In [87]:
import numpy as np
import torch
from torch.utils import data
from torch.nn.functional import normalize
#from d2l import torch as d2l

In [88]:
T=normalize(torch.FloatTensor(np.loadtxt("T.txt")),dim=0)
r_cross=normalize(torch.FloatTensor(np.loadtxt("r_cross.txt")),dim=0)
denorm=torch.norm(torch.FloatTensor(np.loadtxt("r_cross.txt")),p=2,dim=0)

class MyDataset(data.Dataset):
    def __init__(self, feature_data, label_data):
        self.feature_data = feature_data
        self.label_data = label_data

    def __len__(self):
        return len(self.label_data)

    def __getitem__(self, idx):
        return self.feature_data[idx], self.label_data[idx]

dataset = MyDataset(T,r_cross)
train_size=int(0.8*len(r_cross))
test_size=len(r_cross)-train_size
data_train, data_test = data.random_split(dataset, [train_size, test_size])

In [89]:
batch_size = 40
data_iter = data.DataLoader(data_train, batch_size, shuffle=True)
test_data_iter = data.DataLoader(data_test, batch_size, shuffle=False)

In [90]:
next(iter(data_iter))


[tensor([[-0.0683, -0.0669, -0.0688,  ..., -0.0702, -0.0715, -0.0724],
         [-0.0731, -0.0757, -0.0749,  ..., -0.0747, -0.0729, -0.0736],
         [-0.0704, -0.0681, -0.0641,  ..., -0.0650, -0.0670, -0.0638],
         ...,
         [-0.0722, -0.0730, -0.0721,  ..., -0.0729, -0.0724, -0.0743],
         [-0.0706, -0.0681, -0.0665,  ..., -0.0667, -0.0681, -0.0644],
         [-0.0711, -0.0704, -0.0705,  ..., -0.0721, -0.0721, -0.0740]]),
 tensor([0.0929, 0.0667, 0.0402, 0.0674, 0.0882, 0.0375, 0.0368, 0.0817, 0.0439,
         0.0923, 0.0500, 0.0766, 0.0623, 0.0984, 0.0793, 0.0419, 0.0838, 0.0746,
         0.0616, 0.0538, 0.0579, 0.0449, 0.1008, 0.0381, 0.0661, 0.0977, 0.0780,
         0.0572, 0.0436, 0.0718, 0.0895, 0.0851, 0.0940, 0.0344, 0.0497, 0.0412,
         0.0347, 0.0773, 0.0422, 0.0848])]

In [91]:
net=torch.nn.Sequential(torch.nn.Linear(128,128),torch.nn.ReLU(),torch.nn.Linear(128,128),torch.nn.ReLU(),torch.nn.Linear(128,1))

In [96]:
loss = torch.nn.MSELoss()
l = loss(net(T).squeeze(-1), r_cross)
need_l=4e-6
T_test, r_cross_test=next(iter(test_data_iter))
l_test = loss(net(T_test).squeeze(-1), r_cross_test)
while l_test>need_l:

    for i in (0,2,4):
        net[i].weight.data.normal_(0, 0.15)
        net[i].bias.data.fill_(0)

    trainer = torch.optim.SGD(net.parameters(), lr=0.03)

    num_epochs = 100
    for epoch in range(num_epochs):
        for X, y in data_iter:
            l = loss(net(X).squeeze(-1) ,y)
            trainer.zero_grad()
            l.backward()
            trainer.step()
        l = loss(net(T).squeeze(-1), r_cross)
        if l <= need_l:
            break
    print(f'epoch {epoch + 1}, loss {l:f}')
    l_test = loss(net(T_test).squeeze(-1), r_cross_test)
    print(f"test_loss {l_test:f}")



print(net(T_test).squeeze(-1)*denorm*1e4)
print(r_cross_test*denorm*1e4)


epoch 100, loss 0.000036
test_loss 0.000030
epoch 100, loss 0.000008
test_loss 0.000007
epoch 100, loss 0.000060
test_loss 0.000068
epoch 100, loss 0.000042
test_loss 0.000042
epoch 100, loss 0.000050
test_loss 0.000036
epoch 100, loss 0.000024
test_loss 0.000019
epoch 100, loss 0.000006
test_loss 0.000005
epoch 100, loss 0.000038
test_loss 0.000038
epoch 100, loss 0.000006
test_loss 0.000006
epoch 100, loss 0.000020
test_loss 0.000020
epoch 100, loss 0.000010
test_loss 0.000007
epoch 100, loss 0.000027
test_loss 0.000027
epoch 100, loss 0.000012
test_loss 0.000011
epoch 100, loss 0.000011
test_loss 0.000010
epoch 100, loss 0.000019
test_loss 0.000014
epoch 100, loss 0.000007
test_loss 0.000008
epoch 100, loss 0.000086
test_loss 0.000075
epoch 100, loss 0.000019
test_loss 0.000020
epoch 100, loss 0.000063
test_loss 0.000047
epoch 100, loss 0.000019
test_loss 0.000021
epoch 100, loss 0.000052
test_loss 0.000055
epoch 100, loss 0.000011
test_loss 0.000012
epoch 100, loss 0.000033
test_lo